## Crank-Nicosol method on schrodinger equation

In [15]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate
from scipy import sparse
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import eigs



In [16]:
# system parameters 
N = 101
# positive length
L = 10
# grid
x,dx = np.linspace(0,L, N , retstep= True)
# time parameters
T = 0.0001
Nt = 10000
dt = 1e-5

In [17]:
# defining alpha
alpha = (1j * dt) / (4 * (dx**2))

# defining diagonal a and b
a = (1 + (2 * alpha))
b = (1 - (2 * alpha))


# constant matrix :-> MA, MM
def constant_matrix(V):
    dia = np.ones([N])
    dia_A = np.array([dia * -alpha , dia *a, dia * -alpha])
    V0= sparse.diags(-1j * dt *V)
    dia_M = np.array([dia * alpha , dia * b , dia * alpha])
    MA = sparse.spdiags(dia_A,np.array([-1,0,1]),N,N)
    MM = sparse.spdiags(dia_M,np.array([-1,0,1]),N,N)  + V0
    return MA ,MM



In [18]:
def CMN(psi0,V):
    psi = np.zeros([Nt,N]).astype(complex)
    psi[0] = psi0
    MA , MM = constant_matrix(V)
    t = 0 
    y = psi0.copy()
    while t < Nt-1:
        MM = sparse.csr_matrix.dot(MM, y)
        psi_next = sparse.linalg.spsolve(MA, MM)
        psi[t+1] = psi_next
        y = psi_next
        t += 1    
    return psi